In [ ]:
# Infer image using morpheus_core
from morpheus_core import morpheus_core
import torch
import numpy as np
import os
import pickle
import sys
import cv2

# Get the current working directory
current_path = os.getcwd()
print("Current Working Directory:", current_path)
# Get the parent directory
parent_path = os.path.dirname(current_path)
print("Parent Directory:", parent_path)

#sys.path.append(current_path)
sys.path.append(parent_path)

# Print the updated sys.path
print("Updated sys.path:")
for p in sys.path:
    print(p)

# Find available GPU
if torch.backends.mps.is_available(): # Check if PyTorch has access to MPS (Metal Performance Shader, Apple's GPU architecture)
    print("MPS is available!")
    if torch.backends.mps.is_built():
        print("MPS (Metal Performance Shader) is built in!")    
    device = "mps"
elif torch.cuda.is_available(): # Check if PyTorch has access to CUDA (Win or Linux's GPU architecture)
    print("CUDA is available!")
    device = "cuda"
else:
    print("Only CPU is available!")
    device = "cpu"
print(f"Using device: {device}")

#check out kernel
print("Python executable:", sys.executable)
print("Python version:", sys.version)

# Load the pickle file
pickle_file = "../dataloader/annotation/dataset5.pkl"
with open(pickle_file, 'rb') as pkl_file:
    data = pickle.load(pkl_file)  # Load the dictionary
# Retrieve the val_files list
val_files = data['val']
prefix = "../../../../OneDrive/NeuralMorphology/Simulations/Simulations_16bit_Size3334/images/Realistic-SBR-"
updated_files = [prefix + f for f in val_files]
pgm_image = cv2.imread(updated_files[0], cv2.IMREAD_UNCHANGED)  # Load as grayscale
pgm_image = cv2.convertScaleAbs(pgm_image, alpha=255.0 / pgm_image.max()) / 255.
pgm_array = np.array(pgm_image)

from model.unet_att import UnetAttention
model = UnetAttention()
model = model.to(device)
ckpt = torch.load("../../../../OneDrive/NeuralMorphology/Simulations/Simulations_16bit_Size3334/output/ex1/ckpt.pth", weights_only=False, map_location=device)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()  # Set to evaluation mode

model_inputs = pgm_array
n_classes = 1             # number of classes that are output from the model
batch_size = 16           # number of samples to extract per batch
window_shape = (512, 512) # (height, width) of each sample
gpus = [0, 1, 2]          # GPUs to use
cpus = 4                  # number of CPU cores to use for data loading
out_dir = "../../../../OneDrive/NeuralMorphology/Simulations/Simulations_16bit_Size3334/output/ex1/evaluation/"

output_hduls, output_arrays = morpheus_core.predict(
    model,        # your model in a callable from
    model_inputs[:, :, np.newaxis], # list of numpy arrays or strings that point to fits files
    n_classes,
    batch_size,
    window_shape,
)



In [14]:
model_inputs.shape

(3334, 3334)

In [18]:
[oa.shape for oa in output_arrays]

[(3334, 1, 1), (3334, 1)]